<a href="https://colab.research.google.com/github/arkothiwala/QuoraQuestionPairs/blob/master/SnapStanford_Amazon_ElectronicsReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Welcome to Colaboratory!</h1>


Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

With Colaboratory you can write and execute code, save and share your analyses, and access powerful computing resources, all for free from your browser.

In [0]:
# Install latest tensorflow
!pip install tensorflow==2.0.0-beta1 
# Install execution time extension
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable hinterland/hinterland
!jupyter nbextension enable execute_time/ExecuteTime

In [0]:
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz"

In [0]:
import progressbar
pbar = None

def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

In [0]:
import urllib.request

In [5]:
urllib.request.urlretrieve(url,"data.gz", show_progress)

100% (495854086 of 495854086) |##########| Elapsed Time: 0:01:17 Time:  0:01:17


('data.gz', <http.client.HTTPMessage at 0x7f779a627dd8>)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import tensorflow
import time
from cleaning import TextPreProcessing

In [0]:
df = pd.read_json('./data.gz', lines=True, compression='gzip')

In [11]:
%%time
df.isna().sum()

CPU times: user 1.98 s, sys: 3.23 ms, total: 1.98 s
Wall time: 1.98 s


asin                  0
helpful               0
overall               0
reviewText            0
reviewTime            0
reviewerID            0
reviewerName      24730
summary               0
unixReviewTime        0
dtype: int64

In [6]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0528881469,"[0, 0]",5,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1370131200
1,0528881469,"[12, 15]",1,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1290643200
2,0528881469,"[43, 45]",3,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1283990400
3,0528881469,"[9, 10]",2,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1290556800
4,0528881469,"[0, 0]",1,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1317254400


There are lot of columns we do not need. Lets filter our dataset to include only text and rating

In [24]:
%%time
filteredCols = ['reviewText', 'overall']
df = df[filteredCols]

CPU times: user 1.76 s, sys: 14.8 ms, total: 1.77 s
Wall time: 1.77 s


In [0]:
df.head()

In [25]:
%%time
df.reviewText = TextPreProcessing.cleanText(df.reviewText)
df.reviewText = df.reviewText.str.replace(r'([^0-9a-zA-Z ]+)',r' ').str.replace(r"[ ]+"," ")
df.to_csv("data.csv")

cleanText
addStartStopTokens
33.65 % queries with numbers were processed
31.34 % queries with numbers were processed
0.15 % queries with numbers were processed
1.54 % queries with numbers were processed
5.13 % queries with numbers were processed
CPU times: user 9min 16s, sys: 5.27 s, total: 9min 21s
Wall time: 9min 22s


In [0]:
df.to_csv("data.csv")

In [37]:
df.head()

,reviewText,overall
0,start we got this gps for my husband who is a...,5
1,start i is a professional otr truck driver an...,1
2,start well what can i say i is had this unit ...,3
3,start not going to write a long review even t...,2
4,start i is had mine for a year and here is wh...,1


In [0]:
#df.reviewText = TextPreProcessing.removeStartStopTags(df.reviewText, '< start >')
#df.reviewText = print(TextPreProcessing.removeStartStopTags(df.reviewText, '<stop>'))

In [0]:
#https://towardsdatascience.com/yet-another-twitter-sentiment-analysis-part-1-tackling-class-imbalance-4d7a7f717d44

In [5]:
x = ["I'm a professional OTR truck driver, and I bou",
"Well, what can I say. I've had this unit in m...	",
"Not going to write a long review, even thought...	",
"I've had mine for a year and here's what we go...	",
"We got this GPS for my husband who is an (OTR)...	"]

s = pd.Series(x)

TextPreProcessing.cleanText(s)

cleanText
addStartStopTokens
0.00 % queries with numbers were processed
0.00 % queries with numbers were processed
0.00 % queries with numbers were processed
0.00 % queries with numbers were processed
0.00 % queries with numbers were processed


0     < start > i am a professional otr truck drive...
1     < start > well , what can i say . i have had ...
2     < start > not going to write a long review , ...
3     < start > i have had mine for a year and here...
4     < start > we got this gps for my husband who ...
dtype: object

In [0]:
def foo(series):
  #for i in range(len(series)):
  #  series.iloc[i].replace
  #d = dict(zip([r"([a-z]+)'m ",r"([a-z]+)'d "],[r"\1 am ",r"\1 had "]))
  d = dict(zip([r'going',r'year' ],['GOING', "YEAR"]))
  print(d)
  return series.map(d)

In [21]:
foo(s)

{'going': 'GOING', 'year': 'YEAR'}


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
dtype: object

In [6]:
df.head()

,reviewText,overall
0,< start > we got this gps for my husband who ...,5
1,< start > i is a professional otr truck drive...,1
2,"< start > well , what can i say . i is had th...",3
3,"< start > not going to write a long review , ...",2
4,< start > i is had mine for a year and here i...,1
